In [2]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
import torch

In [3]:
train = "/home/DATA/ITWM/Imagenet32x32/Imagenet32_train"
val   = "/home/DATA/ITWM/Imagenet32x32/val_data"

In [4]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

def load_databatch(data_folder, idx, img_size=32):
    data_file = os.path.join(data_folder, 'train_data_batch_')

    d = unpickle(data_file + str(idx))
    x = d['data']
    y = d['labels']
    mean_image = d['mean']

    x = x/np.float32(255)
    mean_image = mean_image/np.float32(255)

    # Labels are indexed from 1, shift it so that indexes start at 0
    y = [i-1 for i in y]
    data_size = x.shape[0]

    x -= mean_image

    img_size2 = img_size * img_size

    x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))
    x = x.reshape((x.shape[0], img_size, img_size, 3)).transpose(0, 3, 1, 2)

    # create mirrored images
    X_train = x[0:data_size, :, :, :]
    Y_train = y[0:data_size]
    X_train_flip = X_train[:, :, :, ::-1]
    Y_train_flip = Y_train
    X_train = np.concatenate((X_train, X_train_flip), axis=0)
    Y_train = np.concatenate((Y_train, Y_train_flip), axis=0)

    return dict(
        X_train=torch.from_numpy(X_train),
        Y_train=torch.from_numpy(Y_train.astype('int32')),
        mean=mean_image)

In [5]:
train_batch = [ train+"/train_data_batch_{}".format(i) for i in range(1, 11)]

In [6]:
train_batch

['/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_1',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_2',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_3',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_4',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_5',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_6',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_7',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_8',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_9',
 '/home/DATA/ITWM/Imagenet32x32/Imagenet32_train/train_data_batch_10']

In [7]:
# Open with Pickle

In [8]:
# pickle.load( open(train_batch[0], 'rb'))

d = unpickle(train_batch[0])

In [9]:
batch.keys()

NameError: name 'batch' is not defined

In [10]:
len(d['labels'])

128116

In [11]:
len(d['data'])

128116

In [12]:
d['data'][0].shape

(3072,)

In [13]:
32 * 32 * 3

3072

In [14]:
x = d['data']
y = d['labels']
mean_image = d['mean']

In [15]:
x = x/np.float32(255)
mean_image = mean_image/np.float32(255)

In [16]:
y = [i-1 for i in y]
data_size = x.shape[0]

In [17]:
x -= mean_image

In [18]:
img_size = 32
img_size2 = img_size * img_size

In [19]:
x = np.dstack((x[:, :img_size2], x[:, img_size2:2*img_size2], x[:, 2*img_size2:]))

In [20]:
x = x.reshape((x.shape[0], img_size, img_size, 3)).transpose(0, 3, 1, 2)

In [21]:
x.shape

(128116, 3, 32, 32)

In [22]:
batch_1 = load_databatch(train, 1, img_size=32)

In [23]:
batch_1.keys()

dict_keys(['X_train', 'Y_train', 'mean'])

In [24]:
batch_1['Y_train'].shape

torch.Size([256232])

In [25]:
batch_1['X_train'].shape

torch.Size([256232, 3, 32, 32])

In [26]:
d_val = unpickle(val)

In [27]:
d_val.keys()

dict_keys(['labels', 'data'])

In [28]:
d_val['data'].shape

(50000, 3072)

In [29]:
d_val['labels'][0]

372

In [31]:
d_val['labels'][:10]

[372, 814, 227, 604, 226, 51, 663, 308, 276, 749]

In [32]:
from datasets import smallimagenet

In [36]:
import torchvision.transforms as transforms
def get_dataloader(dataset_type, root_dir, is_train, batch_size, workers, resolution=32, classes=1000, **kwargs):
    # normalize = transforms.Normalize(mean=kwargs["mean"],
    #                                  std=kwargs["std"])
    normalize = transforms.Normalize(mean=[0.4810, 0.4574, 0.4078], std=[0.2146, 0.2104, 0.2138])

    transformations = [
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ] if is_train else [
        transforms.ToTensor(),
        normalize,
    ]
    trans = transforms.Compose(transformations)
    dataset = smallimagenet.SmallImagenet(root=root_dir, size=resolution, train=is_train, transform=trans,
                                          classes=range(
                                              classes)) if dataset_type == "SmallImageNet" else tinyimagenet.TinyImageNet(
        root=root_dir, train=is_train, transform=trans)
    shuffle = True if is_train else False
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=workers, pin_memory=True)
    return loader

In [37]:
celebahq_validation_loader = get_dataloader("SmallImageNet", '/home/DATA/ITWM/Imagenet32x32', False, 128, 8, 32, 1000)


In [41]:
celebahq_training_loader = get_dataloader("SmallImageNet", '/home/DATA/ITWM/Imagenet32x32'+'/Imagenet32_train', True, 128, 8, 32, 1000)

In [43]:
import pdb;

In [44]:
for (images, labels) in celebahq_training_loader:
    pdb.set_trace()
    

> <ipython-input-44-e851ac83f8ae>(1)<module>()
----> 1 for (images, labels) in celebahq_training_loader:
      2     pdb.set_trace()
      3 



ipdb>  labels[0]


tensor(117)


ipdb>  images.shape


torch.Size([128, 3, 32, 32])


ipdb>  exit


BdbQuit: 

In [ ]:
for (images, labels) in celebahq_validation_loader:
    pdb.set_trace()

> <ipython-input-45-224b29e14c13>(1)<module>()
----> 1 for (images, labels) in celebahq_validation_loader:
      2     pdb.set_trace()



ipdb>  images.shape


torch.Size([128, 3, 32, 32])


ipdb>  labels[0]


tensor(0)


...

> <ipython-input-45-224b29e14c13>(1)<module>()
----> 1 for (images, labels) in celebahq_validation_loader:
      2     pdb.set_trace()



ipdb>  labels


tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7])


ipdb>  c


> <ipython-input-45-224b29e14c13>(1)<module>()
----> 1 for (images, labels) in celebahq_validation_loader:
      2     pdb.set_trace()

